In [ ]:
from build.passwords import password, jupyterhub_user, user
import requests
from requests.auth import HTTPBasicAuth

api_base_url = "https://engr-131-api.eastus.cloudapp.azure.com/"

In [ ]:
params = {"username": "jca92"}
res = requests.get(
    url=api_base_url.rstrip("/") + "/student-grades-testing",
    params=params,
    auth=HTTPBasicAuth(user(), password()),
)

In [ ]:
api_base_url.rstrip("/") + "/student-grades-testing"

In [ ]:
res.text